In [1]:
https://apps.polkcountyiowa.gov/PolkCountyInmates/CurrentInmates/Details?Book_ID=281171
    

SyntaxError: invalid syntax (<ipython-input-1-ded511662324>, line 1)

In [1]:
import requests
from io import BytesIO, SEEK_SET, SEEK_END

class ResponseStream(object):
    def __init__(self, request_iterator):
        self._bytes = BytesIO()
        self._iterator = request_iterator

    def _load_all(self):
        self._bytes.seek(0, SEEK_END)
        for chunk in self._iterator:
            self._bytes.write(chunk)

    def _load_until(self, goal_position):
        current_position = self._bytes.seek(0, SEEK_END)
        while current_position < goal_position:
            try:
                current_position = self._bytes.write(next(self._iterator))
            except StopIteration:
                break

    def tell(self):
        return self._bytes.tell()

    def read(self, size=None):
        left_off_at = self._bytes.tell()
        if size is None:
            self._load_all()
        else:
            goal_position = left_off_at + size
            self._load_until(goal_position)

        self._bytes.seek(left_off_at)
        return self._bytes.read(size)
    
    def seek(self, position, whence=SEEK_SET):
        if whence == SEEK_END:
            self._load_all()
        else:
            self._bytes.seek(position, whence)

In [2]:
def weight_pound_to_kg(pounds):
    return round(0.453592 * int(pounds))

def height_imperial_to_cm(height):
    h_ft = 0
    try:
        h_ft = int(height[:height.find("'")])
        parts = height.split("'")
        inch = ""
        if len(parts) > 1:
            inch = parts[1].strip()
        
        h_inch = int(inch[:inch.find('"')])
    except :
        h_inch = 0
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54)
    return h_cm

def height_imperial_to_cm2(height):
    h_ft = 0
    try:
        h_ft = int(height[0]) 
        h_inch = int(height[1:])
    except :
        h_inch = 0
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54)
    return h_cm

In [3]:
print(weight_pound_to_kg('180'))
print(height_imperial_to_cm2('601'))
print(height_imperial_to_cm2('602'))
print(height_imperial_to_cm2('502'))

82
185
188
157


In [4]:
from bs4 import BeautifulSoup
import requests
from PIL import Image

class PageParser(object):
    
    def __init__(self,entity):
        self.entity = entity
        self.__content = ""
        self.__soup = None

    def base_url(self):
        return "https://apps.polkcountyiowa.gov/PolkCountyInmates/CurrentInmates/"

    def url_for_entity(self):
        return self.base_url() + "Details?Book_ID=" + str(self.entity)
    
    def retrieve_content(self):
        r = requests.get(self.url_for_entity())
        self.__content = r.content
        
    def check(self):
        r = requests.head(self.url_for_entity())
        if r.status_code == 200:
            return True
        return False
        
    def parse_content(self):
        content = self.content
        soup = BeautifulSoup(content, "html.parser")
        if self.is_valid_url(soup):
            self.__soup = soup
            return True
        return False
        
    def is_valid_url(self,soup):
        return soup and "resource cannot be found" not in soup.title.string and \
            not soup(text="An error occurred while processing your request.")
        
    @property   
    def content(self):
        if not self.__content:
            self.retrieve_content()
        return self.__content
    
    @property
    def soup(self):
        if not self.__soup:
            self.parse_content()
        return self.__soup

class PolkCountyInmatesScraper(PageParser):
    
    def __init__(self,entity):
        PageParser.__init__(self,entity)
        
    @classmethod
    def from_entity(cls,entity):
        return cls(entity)
  
    def retrieve_picture_url(self):
        '''
        element <input data-val="true" data-val-number="The field name_id must be a number." 
                    id="Detail_name_id" name="Detail.name_id" type="hidden" value="1200458"/>
        format https://apps.polkcountyiowa.gov/PolkCountyInmates/CurrentInmates/GetInmateImage?nameId=1200458
        '''
        if not self.soup:
            return ""
        input_field = self.soup.find(id="Detail_name_id")
        if input_field:
            name_id = input_field["value"]        
            return self.base_url() + "GetInmateImage?nameId=" + name_id
        return ""
        
    def get_picture(self):
        picture_url = self.retrieve_picture_url()
        if not picture_url:
            return None
        
        r = requests.get(picture_url, stream=True)
        if r.status_code == 200:
            img = Image.open(ResponseStream(r.iter_content(1024)))
            return img
        
        return None
    
    def get_pictures(self):
        pictures = []
        pic = self.get_picture()
        if pic:
            pictures.append(pic)        
        return pictures
    
    def get_labeled_attribute(self,label):
        if not self.soup:
            return ""
        searchable_attr =  label.capitalize()+':_' 
        return self.soup.find('label',attrs={'for':searchable_attr}).nextSibling.strip()
    
    def get_age(self):
        return self.get_labeled_attribute('age')
    
    def get_weight(self):
        weight_pounds = self.get_labeled_attribute('weight')
        return str(weight_pound_to_kg(weight_pounds))
    
    def get_height(self):
        height_pound = self.get_labeled_attribute('height')
        return str(height_imperial_to_cm(height_pound))
    
    def get_race(self):
        return self.get_labeled_attribute('race')
    
    def get_gender(self):
        return self.get_labeled_attribute('sex').lower()
        
    def get_eyes(self):
        return self.get_labeled_attribute('eyes')
    
    def get_hair(self):
        return self.get_labeled_attribute('hair')
    
scaper = PolkCountyInmatesScraper(281171)

In [5]:
class MarionCountyInmatesScraper(PageParser):
    
    def __init__(self,entity):
        PageParser.__init__(self,entity)
        self.details  = None
        
    @classmethod
    def from_entity(cls,entity):
        return cls(entity)
       

    def url_for_entity(self):
        return "https://apps.co.marion.or.us/jailrosters/mccf_roster.html"
    
    def get_details(self):
        if self.details:
            return self.details
        self.details = dict()
        if self.soup:   
            merged = self.soup.find("span", {"id": "dat_{}".format(self.entity)}).text
            elements = merged.strip().split('|')
            self.details["age"] = ""
            self.details["weight"] = str(weight_pound_to_kg(elements[5].strip()))
            self.details["height"] = str(height_imperial_to_cm2(elements[4].strip()))
            self.details["race"] = elements[2].strip()
            self.details["gender"] = elements[3].strip()
            self.details["hair"] = elements[6].strip()
            self.details["eyes"] = elements[7].strip()
        return self.details
  
    def retrieve_picture_url(self):        
        if not self.soup:
            return ""
        return "https://apps.co.marion.or.us/jailrosters/{}.jpg".format(self.entity)
        
        
    def get_picture(self):
        picture_url = self.retrieve_picture_url()
        if not picture_url:
            return None
        
        r = requests.get(picture_url, stream=True)
        if r.status_code == 200:
            img = Image.open(ResponseStream(r.iter_content(1024)))
            return img
        
        return None
    
    def get_pictures(self):
        pictures = []
        pic = self.get_picture()
        if pic:
            pictures.append(pic)        
        return pictures
        
    def get_age(self):
        return self.get_details().get("age","")
    
    def get_weight(self):
        return self.get_details().get("weight","")
    
    def get_height(self):
        return self.get_details().get("height","")
    
    def get_race(self):
        return self.get_details().get("race","")
    
    def get_gender(self):
        return self.get_details().get("gender","")
        
    def get_eyes(self):
        return self.get_details().get("eyes","")
    
    def get_hair(self):
        return self.get_details().get("hair","")
    
scraper = MarionCountyInmatesScraper(68192)
scraper.parse_content()

True

In [6]:
import sqlite3
import os

class MarionCountyInmatesLocalDb(object):
     
        def __init__(self,file_name):
            self.file_name = file_name
            if not os.path.exists(file_name):
                self.init_db()
            self.conn = sqlite3.connect(file_name)
        
        def init_db(self):
            conn = self.get_conn()
            curs = conn.cursor()
            curs.execute('''
            CREATE TABLE inmates (
                entity_id TEXT PRIMARY KEY,
                status TEXT
            )
            ''')
            conn.commit()
            conn.close()
        
        def get_conn(self):
            return sqlite3.connect(self.file_name)
        
        def get_cursor(self):
            return self.conn.cursor()
        
        def is_new(self,entity):
            cursor = self.get_cursor()
            query = "SELECT count(*) FROM inmates WHERE entity_id="+str(entity)
            cursor.execute(query)
            result = cursor.fetchone()
            return result[0] == 0
            
        def add_new(self,entity):
            conn = self.get_conn()
            cursor = conn.cursor()
            query = "INSERT INTO inmates VALUES ({},'pending')".format(entity)
            try:
                cursor.execute(query)
            except:
                pass
            finally:
                conn.commit()
                conn.close()
        
        def get_next_entity(self):
            cursor = self.get_cursor()
            query = "SELECT entity_id FROM inmates WHERE status='pending'"
            cursor.execute(query)
            result = cursor.fetchone()
            if result:
                return result[0]
            return None
            
        def mark_done(self,entity,reason):
            conn = self.get_conn()
            cursor = conn.cursor()
            query = "UPDATE inmates SET status='{}' WHERE entity_id={}".format(reason,entity)
            try:
                cursor.execute(query)
            except Exception as e:
                print(e)
            finally:
                conn.commit()
                conn.close()
                
        def print_all(self):
            try:
                conn = self.get_conn()
                cursor = conn.cursor()
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
                tables = cursor.fetchall()
                for tbl in tables:
                    print("\n########  "+tbl[0]+"  ########")
                    cursor.execute("SELECT * FROM "+tbl[0]+";")
                    rows = cursor.fetchall()
                    for row in rows:
                        print(row)
            except KeyboardInterrupt:
                pass
              
        
        
    
        def __del__(self):
            if self.conn:
                self.conn.close()
                
        def __enter__(self):
            return self
        
        def __exit__(self,exc_type, exc_value, traceback):
            if self.conn:
                self.conn.close()

dbMarionCounty = MarionCountyInmatesLocalDb('MarionCountyInmates.db')

In [7]:
class MarionCountyInmatesCrawler(object):
    
    def __init__(self,db):
        self.db = db
        
  
    def retrieve_list(self):  
        url = 'https://apps.co.marion.or.us/jailrosters/mccf_roster.html'
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, "html.parser")
            odd_row = soup.findAll("tr", {"style": "display:none;"})
            for row in odd_row:
                entity_id = row["id"].split('_')[1]
                if self.db.is_new(entity_id):
                    self.db.add_new(entity_id)
            
                        
dbMarionCounty = MarionCountyInmatesLocalDb('MarionCountyInmates.db')
crawlerMarionCounty = MarionCountyInmatesCrawler(dbMarionCounty)

In [8]:
crawlerMarionCounty.retrieve_list()

In [9]:
dbMarionCounty.print_all()


########  inmates  ########
('628907', 'processed')
('68192', 'processed')
('327549', 'processed')
('44987', 'processed')
('22566', 'processed')
('727516', 'processed')
('682474', 'processed')
('512311', 'processed')
('19632', 'processed')
('16977', 'processed')
('710959', 'processed')
('683822', 'processed')
('722359', 'processed')
('63679', 'processed')
('62033', 'processed')
('733409', 'processed')
('730220', 'processed')
('156541', 'processed')
('44668', 'processed')
('40458', 'processed')
('422484', 'processed')
('712820', 'processed')
('400571', 'processed')
('265770', 'processed')
('619680', 'processed')
('733865', 'processed')
('679556', 'processed')
('443019', 'processed')
('485468', 'processed')
('692201', 'processed')
('538283', 'processed')
('43050', 'processed')
('729673', 'processed')
('70268', 'processed')
('124062', 'processed')
('463045', 'processed')
('675722', 'processed')
('652885', 'processed')
('624441', 'processed')
('704741', 'processed')
('728392', 'processed'

In [16]:
entity = dbMarionCounty.get_next_entity()
while entity:
    print(entity)
    
    scrapper = Scrapper(entity, MarionCountyInmatesScraper)
    try:
        scrapper.scrape()
        person = scrapper.person        
    except:
        print("failed")
        person = None
    
    
    if not person:
        dbMarionCounty.mark_done(entity,'failed')
    else:
        bmiLocalDB.add_new(person)
        dbMarionCounty.mark_done(entity,'processed')
        
    entity = dbMarionCounty.get_next_entity()

737771
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F15A6BA8>]
737913
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F103D978>]
373802
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F29849E8>]
730911
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F10FCA20>]
59565
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F2D88A58>]
57848
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F117EA90>]
549542
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F18BFAC8>]
572872
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F2FCBAC8>]
127361
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F127EB38>]
661602
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1B3EB70>]
533413
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F31CDBA8>]
4

337591
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1702CC0>]
659836
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1881CF8>]
381406
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1CC0D30>]
128189
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F16C9D68>]
293616
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1E01F98>]
342223
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1FC3978>]
324135
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F11009E8>]
117872
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F2E89A20>]
709687
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F10C1A58>]
144357
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F310DA90>]
322409
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x600 at 0x7F78F1381AC8>]

In [10]:
import sqlite3
import os

class PolkCountyInmatesLocalDb(object):
     
        def __init__(self,file_name):
            self.file_name = file_name
            if not os.path.exists(file_name):
                self.init_db()
            self.conn = sqlite3.connect(file_name)
        
        def init_db(self):
            conn = self.get_conn()
            curs = conn.cursor()
            curs.execute('''
            CREATE TABLE inmates (
                entity_id TEXT PRIMARY KEY,
                status TEXT
            )
            ''')
            conn.commit()
            conn.close()
        
        def get_conn(self):
            return sqlite3.connect(self.file_name)
        
        def get_cursor(self):
            return self.conn.cursor()
        
        def is_new(self,entity):
            cursor = self.get_cursor()
            query = "SELECT count(*) FROM inmates WHERE entity_id="+str(entity)
            cursor.execute(query)
            result = cursor.fetchone()
            return result[0] == 0
            
        def add_new(self,entity):
            conn = self.get_conn()
            cursor = conn.cursor()
            query = "INSERT INTO inmates VALUES ({},'pending')".format(entity)
            try:
                cursor.execute(query)
            except:
                pass
            finally:
                conn.commit()
                conn.close()
        
        def get_next_entity(self):
            cursor = self.get_cursor()
            query = "SELECT entity_id FROM inmates WHERE status='pending'"
            cursor.execute(query)
            result = cursor.fetchone()
            if result:
                return result[0]
            return None
            
        def mark_done(self,entity,reason):
            conn = self.get_conn()
            cursor = conn.cursor()
            query = "UPDATE inmates SET status='{}' WHERE entity_id={}".format(reason,entity)
            try:
                cursor.execute(query)
            except Exception as e:
                print(e)
            finally:
                conn.commit()
                conn.close()
                
        def print_all(self):
            try:
                conn = self.get_conn()
                cursor = conn.cursor()
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
                tables = cursor.fetchall()
                for tbl in tables:
                    print("\n########  "+tbl[0]+"  ########")
                    cursor.execute("SELECT * FROM "+tbl[0]+";")
                    rows = cursor.fetchall()
                    for row in rows:
                        print(row)
            except KeyboardInterrupt:
                pass
              
        
        
    
        def __del__(self):
            if self.conn:
                self.conn.close()
                
        def __enter__(self):
            return self
        
        def __exit__(self,exc_type, exc_value, traceback):
            if self.conn:
                self.conn.close()

db = PolkCountyInmatesLocalDb('PolkCountyInmates.db')
print(db.add_new('281172'))
print(db.is_new('281172'))
print(db.get_next_entity())
#db.mark_done('281172')

None
False
None


In [11]:
class PolkCountyInmatesCrawler(object):
    
    def __init__(self,db):
        self.db = db
        
  
    def retrieve_list(self):  
        url = 'https://apps.polkcountyiowa.gov/PolkCountyInmates/CurrentInmates/JsonList'
        r = requests.post(url, stream=True)
        if r.status_code == 200:
            for inmate in r.json()["data"]:
                if self.db.is_new(inmate["book_id"]):
                    self.db.add_new(inmate["book_id"])
                        
db = PolkCountyInmatesLocalDb('PolkCountyInmates.db')    
crawler = PolkCountyInmatesCrawler(db)

In [12]:
class Person(object):
    
    def __init__(self):
        self.pictures = []
        self.height = 0
        self.weight = 0
        self.gender = ""
        self.race = ""
        self.eyes = ""
        self.hair = ""
        self.unknown = None

In [13]:
class Scrapper(object):
    
    def __init__(self, entity, scrapper):
        self.scrapper = scrapper.from_entity(entity)
        self.person = Person()
        
    def scrape(self):
        if not self.scrapper.parse_content():
            self.person = None
            return 
        for key,value in self.person.__dict__.items():
             
            try:
                method_to_call = getattr(self.scrapper,'get_' + key)
            except:
                 method_to_call = None
            if method_to_call:
                value = method_to_call()
                if isinstance(value,int):
                    setattr(self.person, key, int(value))
                else:
                    setattr(self.person, key, value)
                
            

In [14]:
import sqlite3
import os

class BMILocalDb(object):
     
        def __init__(self,file_name,folder):
            self.file_name = file_name
            if not os.path.exists(file_name):
                self.init_db()
            self.conn = sqlite3.connect(file_name)
            self.folder = folder
        
        def init_db(self):
            conn = self.get_conn()
            curs = conn.cursor()
            curs.execute('''
            CREATE TABLE bmi (
                pictures_path TEXT,
                height DOUBLE,
                weight DOU,
                gender TEXT,
                race TEXT,
                eyes TEXT,
                hair TEXT
            )
            ''')
            conn.commit()
            conn.close()
        
        def get_conn(self):
            return sqlite3.connect(self.file_name)
        
        def get_cursor(self):
            return self.conn.cursor()
        
     
        def save_picture(self, person, pic_id):
            pic_path = str(pic_id) +".png"
            print(person.pictures)
            person.pictures[0].save(os.path.join(self.folder,pic_path),"PNG")
            return pic_path
            
        def add_new(self,person):
            conn = self.get_conn()
            cursor = conn.cursor()
            query = "INSERT INTO bmi VALUES ('{}',{},{},'{}','{}','{}','{}')".format('',person.height, person.weight, person.gender, \
                                                                             person.race,person.eyes,person.hair)
            
            try:
                cursor.execute(query)
                pic_id = cursor.lastrowid
                pic_path = self.save_picture(person,pic_id)
                update_pic = "UPDATE bmi SET pictures_path='{}' WHERE rowid={}".format(pic_path,pic_id)
                cursor.execute(update_pic)
            except Exception as e:
                print(e)
            finally:
                conn.commit()
                conn.close()
        
                
        def print_all(self):
            try:
                conn = self.get_conn()
                cursor = conn.cursor()
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
                tables = cursor.fetchall()
                for tbl in tables:
                    print("\n########  "+tbl[0]+"  ########")
                    cursor.execute("SELECT * FROM "+tbl[0]+";")
                    rows = cursor.fetchall()
                    for row in rows:
                        print(row)
            except KeyboardInterrupt:
                pass
            
        def __del__(self):
            if self.conn:
                self.conn.close()
                
        def __enter__(self):
            return self
        
        def __exit__(self,exc_type, exc_value, traceback):
            if self.conn:
                self.conn.close()

In [15]:
bmiLocalDB = BMILocalDb('BMI.db','images')

In [17]:
crawler.retrieve_list()


In [18]:

entity = db.get_next_entity()
while entity:
    print(entity)
    scrapper = Scrapper(entity, PolkCountyInmatesScraper)
    scrapper.scrape()
    person = scrapper.person
    
    if not person:
        db.mark_done(entity,'failed')
    else:
        bmiLocalDB.add_new(person)
        db.mark_done(entity,'processed')
        
    entity = db.get_next_entity()
bmiLocalDB.print_all()   

286788
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1E20E10>]
286787
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1E02EF0>]
286786
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1E27550>]
286785
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1D5F6D8>]
286777
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DBCD30>]
286775
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1C1DEB8>]
286774
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1C025F8>]
286771
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1C02390>]
286769
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1E02C50>]
286763
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DBC7F0>]
286761
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1E02C50>]

286559
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F10EB438>]
286551
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1B25208>]
286548
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1354B70>]
286547
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1BFD6D8>]
286545
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DCA6D8>]
286544
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DE6470>]
286543
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1D4EE80>]
286540
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1D4EBE0>]
286539
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F10F0358>]
286538
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1AAA160>]
286535
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1D2DEF0>]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DB3940>]
286264
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0BBAA20>]
286261
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0C16CC0>]
286258
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0BBA1D0>]
286245
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1ACD198>]
286238
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DACB00>]
286237
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DAC400>]
286235
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DCA208>]
286234
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1DAC2E8>]
286230
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1AAA550>]
286229
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0B00518>]
286228

285844
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F10F7EB8>]
285829
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0BBA1D0>]
285825
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0BBA240>]
285801
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1B25550>]
285787
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1A67EF0>]
285785
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1AFAFD0>]
285783
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1A2B780>]
285774
[]
list index out of range
285763
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F1AC2550>]
285750
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F0A91E80>]
285744
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F78F09E3080>]
285737
[<PIL.JpegImagePlugin.JpegImageFile image mode=RG

In [ ]:
scrapper = Scrapper(281171, PolkCountyInmatesScraper)
scrapper.scrape()

In [ ]:
count = 0
for i in range(281172,1,-1):
    scrapper = PolkCountyInmatesScraper(i)
    if scrapper.check():
        count = count + 1
    if i % 100 == 0:
        print( str(i) + "->" + str(count))
        

In [ ]:
print(soup)

In [128]:
scaper.get_height()

5
10


'178'

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=720x480 at 0x7F4BFE38DC18>


In [140]:
import pandas as pd
db_bmi = sqlite3.connect('BMI.db')
table_name = 'bmi'
table = pd.read_sql_query("SELECT * from %s" % table_name, db_bmi)
table.to_csv(table_name + '.csv', index_label='index')

In [75]:
db.print_all() 


########  inmates  ########
('283159', 'processed')
('283158', 'processed')
('283156', 'failed')
('283155', 'processed')
('283154', 'processed')
('283153', 'processed')
('283152', 'processed')
('283151', 'processed')
('283149', 'processed')
('283145', 'processed')
('283139', 'processed')
('283125', 'processed')
('283123', 'processed')
('283114', 'processed')
('283110', 'processed')
('283104', 'processed')
('283099', 'processed')
('283091', 'processed')
('283082', 'processed')
('283067', 'processed')
('283066', 'processed')
('283065', 'processed')
('283063', 'processed')
('283062', 'processed')
('283060', 'processed')
('283059', 'processed')
('283058', 'processed')
('283057', 'processed')
('283056', 'processed')
('283042', 'processed')
('283041', 'processed')
('283040', 'processed')
('283038', 'processed')
('283028', 'processed')
('283026', 'processed')
('283025', 'processed')
('283024', 'processed')
('283023', 'processed')
('283020', 'processed')
('283019', 'processed')
('283015', 'pr

In [74]:
conn = sqlite3.connect('PolkCountyInmates.db')
curs = conn.cursor()
curs.execute('''
            DELETE FROM inmates WHERE status='pending'
            ''')
conn.commit()
conn.close()